<a href="https://colab.research.google.com/github/deepsyyt/ML-problem/blob/master/ML_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Selecting only recommended airport codes and year

In [ ]:
import os
import shutil
import pandas as pd
import json

In [ ]:
# List of airport codes
airport_codes = ["ATL","CLT","DEN","DFW","EWR","IAH","JFK","LAS","LAX","MCO",
"MIA","ORD","PHX","SEA","SFO"]  # Add all your airport codes

In [ ]:

base_path = '/content/drive/MyDrive/ML project/weather'
destination_path = '/content/drive/MyDrive/ML project/selected_weather'

# selected_years
selected_years = ["2016", "2017"]

for airport_code in airport_codes:
    airport_folder = os.path.join(base_path, airport_code)
    for year in selected_years:
        for month in range(1, 13):
            json_file_name = f"{year}-{month}.json"
            json_file_path = os.path.join(airport_folder, json_file_name)
            # Checking if the file exists before copying
            if os.path.exists(json_file_path):
                # Destination folder for the selected years
                destination_folder = os.path.join(destination_path, airport_code, year)
                # Create the destination folder if it doesn't exist
                os.makedirs(destination_folder, exist_ok=True)
                # Copying the file to the destination
                shutil.copy(json_file_path, destination_folder)
print("copy completed")

copy completed


Selecting recommended columns and rows from the Json file of each airport_code

In [ ]:
selected_columns_weather = ["windspeedKmph","winddirDegree","weatherCode","precipMM",
"visibility","pressure","cloudcover","DewPointF","WindGustKmph","tempF","WindChillF",
"humidity","date","time","airport"]

# DataFrame to store all weather data
final_weather_df = []

base_path_selected_years = '/content/drive/MyDrive/ML project/selected_weather'
# Iterating through each airport, year, and month
for airport_code in os.listdir(base_path_selected_years):
    airport_folder = os.path.join(base_path_selected_years, airport_code)

    for year in os.listdir(airport_folder):
        year_folder = os.path.join(airport_folder, year)

        for month in range(1, 13):
            json_file_name = f"{year}-{month}.json"
            json_file_path = os.path.join(year_folder, json_file_name)

            if os.path.exists(json_file_path):
                # Reading JSON file
                with open(json_file_path, 'r') as f:
                    data = json.load(f)

            # Extracting data from "weather" key
            weather_data = data.get('data', {}).get('weather', [])

            # Extracting data from "request" key
            request_data = data.get('data', {}).get('request', [{}])[0]

            # Normalizing the JSON data using pandas.json_normalize
            df_weather = pd.json_normalize(weather_data, 'hourly', ['date'])

            # Adding the "query" information to the DataFrame
            df_weather['airport'] = request_data.get('query', None)
            selected_weather_data = df_weather[selected_columns_weather]
            # Appending the DataFrame to the list
            final_weather_df.append(selected_weather_data)

# Concatenating all DataFrames in the list
final_weather_df = pd.concat(final_weather_df, ignore_index=True)
final_weather_df['airport'] = final_weather_df['airport'].str.extract(r'([A-Z]{3})')
print(final_weather_df.shape)
# copying  the resulting DataFrame
final_weather_df.to_csv('/content/drive/MyDrive/ML project/final_weather_df.csv', index=False)

(263160, 15)


In [ ]:
# the CSV file import
final_weather_df = pd.read_csv('/content/drive/MyDrive/ML project/final_weather_df.csv')

In [ ]:
final_weather_df.head()

,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,date,time,airport
0,11,318,176,0.1,10,1023,100,47,17,49,46,91,2016-01-01,0,ATL
1,13,317,176,0.0,10,1023,100,44,22,46,42,92,2016-01-01,100,ATL
2,14,315,122,0.0,10,1023,100,41,26,43,38,92,2016-01-01,200,ATL
3,16,314,122,0.0,10,1023,100,38,30,40,33,93,2016-01-01,300,ATL
4,17,314,122,0.0,10,1023,100,38,30,40,33,93,2016-01-01,400,ATL


In [ ]:
final_weather_df['time'].value_counts()

0       10965
100     10965
2200    10965
2100    10965
2000    10965
1900    10965
1800    10965
1700    10965
1600    10965
1500    10965
1400    10965
1300    10965
1200    10965
1100    10965
1000    10965
900     10965
800     10965
700     10965
600     10965
500     10965
400     10965
300     10965
200     10965
2300    10965
Name: time, dtype: int64

In [ ]:
final_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263160 entries, 0 to 263159
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   windspeedKmph  263160 non-null  int64  
 1   winddirDegree  263160 non-null  int64  
 2   weatherCode    263160 non-null  int64  
 3   precipMM       263160 non-null  float64
 4   visibility     263160 non-null  int64  
 5   pressure       263160 non-null  int64  
 6   cloudcover     263160 non-null  int64  
 7   DewPointF      263160 non-null  int64  
 8   WindGustKmph   263160 non-null  int64  
 9   tempF          263160 non-null  int64  
 10  WindChillF     263160 non-null  int64  
 11  humidity       263160 non-null  int64  
 12  date           263160 non-null  object 
 13  time           263160 non-null  int64  
 14  airport        263160 non-null  object 
dtypes: float64(1), int64(12), object(2)
memory usage: 30.1+ MB


In [ ]:
final_weather_df.isna().sum()

windspeedKmph    0
winddirDegree    0
weatherCode      0
precipMM         0
visibility       0
pressure         0
cloudcover       0
DewPointF        0
WindGustKmph     0
tempF            0
WindChillF       0
humidity         0
date             0
time             0
airport          0
dtype: int64

Unzipping flight data

In [ ]:
import zipfile
import os

# creating the path to your zip file flight data
zip_file_flight_path = '/content/drive/MyDrive/ML project/Flight Data'

# the destination folder to unzip the data
extracted_folder_path_flight = '/content/drive/MyDrive/ML project/unzipped_flight_data'

# Creating the destination folder
os.makedirs(extracted_folder_path_flight, exist_ok=True)

 # Listing all zip files in the folder
zip_files = [file for file in os.listdir(zip_file_flight_path)]
# Unzippiing each file
for zip_file in zip_files:
    zip_file_path = os.path.join(zip_file_flight_path, zip_file)
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
         zip_ref.extractall(extracted_folder_path_flight)

# Checking the contents of the extracted folder
extracted_files = os.listdir(extracted_folder_path_flight)
print(extracted_files)

['2017', '2016']


Selecting only recommended columns from the flight data

In [ ]:
select_flight_data_columns = ["Origin","FlightDate","Quarter","Year","Month",
"DayofMonth","DepTime","DepDel15","CRSDepTime","DepDelayMinutes",
"OriginAirportID","DestAirportID","ArrTime","CRSArrTime","ArrDel15",
"ArrDelayMinutes"]

Destination_id = [10397,11057,11292,11298,11618,12266,12478,12889,12892,
13204,13303,13930,14107,14747,14771]

extracted_folder_path_flight = '/content/drive/MyDrive/ML project/unzipped_flight_data'
# Iterating through the years
all_dataframes=[]
for year in os.listdir(extracted_folder_path_flight):
    path_year_folder = os.path.join(extracted_folder_path_flight, year)
    #Recursively iterating through all subdirectories and files
    for root, dirs, files in os.walk(path_year_folder):
        for file_name in files:
        # Checking if the file is a CSV file
            if file_name.endswith('.csv'):
            # Constructing the full path to the CSV file
               csv_path = os.path.join(root, file_name)
               # Reading the CSV file into a DataFrame
               df_flight = pd.read_csv(csv_path, usecols=select_flight_data_columns)
               #print(df_flight.shape)
               all_dataframes.append(df_flight)

flight_final_df = pd.concat(all_dataframes, ignore_index=True)
condition_origin = flight_final_df['Origin'].isin(airport_codes)
flight_final_df = flight_final_df[condition_origin]
condition_destination = flight_final_df['DestAirportID'].isin(Destination_id)
flight_final_df = flight_final_df[condition_destination]
print(flight_final_df.shape)
flight_final_df.head()

(1877296, 16)


,Year,Quarter,Month,DayofMonth,FlightDate,OriginAirportID,Origin,DestAirportID,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15
0,2017,1,1,17,2017-01-17,11057,CLT,14107,1619,1616.0,0.0,0.0,1856,1842.0,0.0,0.0
1,2017,1,1,18,2017-01-18,11057,CLT,14107,1619,1614.0,0.0,0.0,1856,1821.0,0.0,0.0
2,2017,1,1,19,2017-01-19,11057,CLT,14107,1619,1611.0,0.0,0.0,1856,1826.0,0.0,0.0
3,2017,1,1,20,2017-01-20,11057,CLT,14107,1619,1656.0,37.0,1.0,1856,1929.0,33.0,1.0
4,2017,1,1,21,2017-01-21,11057,CLT,14107,1619,1632.0,13.0,0.0,1856,1858.0,2.0,0.0


In [ ]:
flight_final_df["Origin"].value_counts()

LAX    212087
SFO    169580
ORD    155377
ATL    148914
DEN    145885
LAS    133745
DFW    127491
SEA    116778
PHX    115102
EWR    102025
MCO    101370
JFK     94976
IAH     91310
CLT     84686
MIA     77970
Name: Origin, dtype: int64

In [ ]:
flight_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5055659 entries, 0 to 5055658
Data columns (total 16 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Year             int64  
 1   Quarter          int64  
 2   Month            int64  
 3   DayofMonth       int64  
 4   FlightDate       object 
 5   OriginAirportID  int64  
 6   Origin           object 
 7   DestAirportID    int64  
 8   CRSDepTime       int64  
 9   DepTime          float64
 10  DepDelayMinutes  float64
 11  DepDel15         float64
 12  CRSArrTime       int64  
 13  ArrTime          float64
 14  ArrDelayMinutes  float64
 15  ArrDel15         float64
dtypes: float64(6), int64(8), object(2)
memory usage: 617.1+ MB


In [ ]:
flight_final_df.isna().sum()

Year                   0
Quarter                0
Month                  0
DayofMonth             0
FlightDate             0
OriginAirportID        0
Origin                 0
DestAirportID          0
CRSDepTime             0
DepTime            20980
DepDelayMinutes    20987
DepDel15           20987
CRSArrTime             0
ArrTime            22080
ArrDelayMinutes    25860
ArrDel15           25860
dtype: int64

In [ ]:
flight_df = flight_final_df.dropna()

In [ ]:
flight_df.isna().sum()

Year               0
Quarter            0
Month              0
DayofMonth         0
FlightDate         0
OriginAirportID    0
Origin             0
DestAirportID      0
CRSDepTime         0
DepTime            0
DepDelayMinutes    0
DepDel15           0
CRSArrTime         0
ArrTime            0
ArrDelayMinutes    0
ArrDel15           0
dtype: int64

In [ ]:
flight_df.shape

(1851436, 16)

In [ ]:
def round_arrival_time(time):
    if pd.isna(time):
        return None
    if isinstance(time, float):
        # Round up if decimal part is 0.5 or greater
        if time - int(time) >= 0.5:
            time = int(time) + 1
        else:
            time = int(time)
    if not str(time):
        return None
    # Convert to integer to handle cases like 23.0
    time = int(time)
    time_str = str(time)
    rounded_time = int(time_str[:2]) * 100 + 100 if len(time_str) >= 3 and int(time_str[2:]) > 30 else int(time_str[:2]) * 100
    return rounded_time

# Apply the rounding logic to the ArrTime column
flight_df['Approximate_arrival_time'] = flight_df['ArrTime'].apply(round_arrival_time)

<ipython-input-28-06ae51fc246b>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flight_df['Approximate_arrival_time'] = flight_df['ArrTime'].apply(round_arrival_time)


In [ ]:
flight_df["Approximate_arrival_time"].value_counts()

2100    117840
1900    114958
2000    113455
1700    112587
1800    110123
         ...  
4900       593
5700       521
5600       484
5800       482
5900       474
Name: Approximate_arrival_time, Length: 83, dtype: int64

In [ ]:
flight_df.to_csv('/content/drive/MyDrive/ML project/flight_df.csv')

In [ ]:
# the CSV file download
flight_df = pd.read_csv('/content/drive/MyDrive/ML project/flight_df.csv')

In [ ]:
flight_df.shape

(1851436, 17)

In [ ]:
# Group by 'OriginAirportID' and 'Origin', then counting the occurrences
distinct_counts = flight_df.groupby(['OriginAirportID', 'Origin']).size().reset_index(name='Counts')

# Display the resulting DataFrame
display(distinct_counts)

,OriginAirportID,Origin,Counts
0,10397,ATL,147065
1,11057,CLT,83486
2,11292,DEN,144149
3,11298,DFW,125904
4,11618,EWR,100170
5,12266,IAH,89551
6,12478,JFK,93437
7,12889,LAS,132155
8,12892,LAX,209322
9,13204,MCO,99568


In [ ]:
# Mergeing based on Origin",'FlightDate"
merged_df = pd.merge(flight_df, final_weather_df, left_on=["Origin",'FlightDate','Approximate_arrival_time'], right_on=["airport",'date','time'], how='left')
print(f"{merged_df.shape} merged df rows and columns")
merged_df.head()

(1851436, 33) merged df rows and columns


,Unnamed: 0,Year,Quarter,Month,DayofMonth,FlightDate,OriginAirportID,Origin,DestAirportID,CRSDepTime,...,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,date,time,airport
0,0,2017,1,1,17,2017-01-17,11057,CLT,14107,1619,...,1020.0,93.0,57.0,22.0,63.0,62.0,83.0,2017-01-17,1900.0,CLT
1,1,2017,1,1,18,2017-01-18,11057,CLT,14107,1619,...,1018.0,2.0,43.0,19.0,60.0,58.0,55.0,2017-01-18,1800.0,CLT
2,2,2017,1,1,19,2017-01-19,11057,CLT,14107,1619,...,1016.0,40.0,46.0,6.0,59.0,59.0,64.0,2017-01-19,1800.0,CLT
3,3,2017,1,1,20,2017-01-20,11057,CLT,14107,1619,...,1010.0,61.0,59.0,8.0,65.0,64.0,83.0,2017-01-20,1900.0,CLT
4,4,2017,1,1,21,2017-01-21,11057,CLT,14107,1619,...,1007.0,100.0,57.0,11.0,61.0,60.0,91.0,2017-01-21,1900.0,CLT
